# Data Formatting

## Téléchargement des données de la SRTM

Il nous a fallu dans un premier temps télécharger les données cartographiques de la SRTM, l'interface de leur site n'étant pas adapté à un téléchargement massif de données, nous avons développé un script permettant d'automatiser les téléchargements. 

In [ ]:
import requests
import os

for i in range(1, 72):
    for j in range(1, 24):
        if os.path.isfile("../downloaded/srtm_{}_{}.zip".format(str(i).zfill(2), str(j).zfill(2))):
            print("[Found] srtm_{}_{}.zip".format(str(i).zfill(2), str(j).zfill(2)))
        else:
            response = requests.get("http://srtm.csi.cgiar.org/wp-content/uploads/files/srtm_5x5/TIFF/srtm_{}_{}.zip".format(str(i).zfill(2), str(j).zfill(2)))
            if response.status_code == 200:
                file_name = "srtm_{}_{}.zip".format(str(i).zfill(2), str(j).zfill(2))
                file_path = "../downloaded/" + file_name
                with open(file_path, 'wb') as f:
                    print("[Downloaded]: " + file_name)
                    f.write(response.content)
            else:
                print("[Not Found] srtm_{}_{}.zip".format(str(i).zfill(2), str(j).zfill(2)))

Une fois ces données téléchargées, il a fallu créer un extracteur pour récupérer uniquement nos fichiers .tif

In [ ]:
import os
import fnmatch
import zipfile


def extractor(zip_file):
    extensions = '.tif'
    [zip_file.extract(file, r"../downloaded/tif") for file in zip_file.namelist() if file.endswith(extensions)]


def extractTifFiles():
    if not os.path.exists("../downloaded/tif"):
        os.makedirs("../downloaded/tif")
    pattern = '*.zip'
    for root, dirs, files in os.walk('../downloaded'):
        for filename in fnmatch.filter(files, pattern):
            if os.path.isfile(os.path.join('../downloaded/tif/', os.path.splitext(filename)[0] + '.tif')):
                print(os.path.splitext(filename)[0] + '.tif trouvé')
                continue
            with zipfile.ZipFile(os.path.join(root, filename)) as zf:
                extractor(zf)
                print(filename, "extrait!")


if __name__ == "__main__":
    extractTifFiles()

Enfin, après avoir extrait nos images, il convenait de les réduire de manière à pouvoir les utiliser dans notre classificateur, nous avons donc subdivisé nos images d'une taille de 6000x6000px en 100 images plus petites d'une taille de 600x600px à l'aide de la librairie GDAL permettant de faire des opérations sur des GeoTIFF.

In [ ]:
import os
from osgeo import gdal, osr


def get_extent(dataset):
    cols = dataset.RasterXSize
    rows = dataset.RasterYSize
    transform = dataset.GetGeoTransform()
    minx = transform[0]
    maxx = transform[0] + cols * transform[1] + rows * transform[2]

    miny = transform[3] + cols * transform[4] + rows * transform[5]
    maxy = transform[3]

    return {
        "minX": str(minx), "maxX": str(maxx),
        "minY": str(miny), "maxY": str(maxy),
        "cols": str(cols), "rows": str(rows)
    }


def create_tiles(minx, miny, maxx, maxy, n):
    width = maxx - minx
    height = maxy - miny

    matrix = []

    for j in range(n, 0, -1):
        for i in range(0, n):
            ulx = minx + (width / n) * i  # 10/5 * 1
            uly = miny + (height / n) * j  # 10/5 * 1

            lrx = minx + (width / n) * (i + 1)
            lry = miny + (height / n) * (j - 1)
            matrix.append([[ulx, uly], [lrx, lry]])

    return matrix


def split(file_name, n):
    raw_file_name = os.path.splitext(os.path.basename(file_name))[0].replace("_downsample", "")
    driver = gdal.GetDriverByName('GTiff')
    dataset = gdal.Open(file_name)
    band = dataset.GetRasterBand(1)
    transform = dataset.GetGeoTransform()

    extent = get_extent(dataset)

    cols = int(extent["cols"])
    rows = int(extent["rows"])

    minx = float(extent["minX"])
    maxx = float(extent["maxX"])
    miny = float(extent["minY"])
    maxy = float(extent["maxY"])

    width = maxx - minx
    height = maxy - miny

    output_path = os.path.join("../downloaded/subdivided", raw_file_name)
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    tiles = create_tiles(minx, miny, maxx, maxy, n)
    transform = dataset.GetGeoTransform()
    xOrigin = transform[0]
    yOrigin = transform[3]
    pixelWidth = transform[1]
    pixelHeight = -transform[5]

    tile_num = 1
    for tile in tiles:
        minx = tile[0][0]
        maxx = tile[1][0]
        miny = tile[1][1]
        maxy = tile[0][1]

        p1 = (minx, maxy)
        p2 = (maxx, miny)

        i1 = int((p1[0] - xOrigin) / pixelWidth)
        j1 = int((yOrigin - p1[1]) / pixelHeight)
        i2 = int((p2[0] - xOrigin) / pixelWidth)
        j2 = int((yOrigin - p2[1]) / pixelHeight)

        new_cols = i2 - i1
        new_rows = j2 - j1

        data = band.ReadAsArray(i1, j1, new_cols, new_rows)

        # print data

        new_x = xOrigin + i1 * pixelWidth
        new_y = yOrigin - j1 * pixelHeight

        new_transform = (new_x, transform[1], transform[2], new_y, transform[4], transform[5])

        output_file_base = raw_file_name + "_" + str(tile_num) + ".tif"
        output_file = os.path.join("../downloaded/subdivided", raw_file_name, output_file_base)

        dst_ds = driver.Create(output_file,
                               new_cols,
                               new_rows,
                               1,
                               gdal.GDT_Float32)

        # writting output raster
        dst_ds.GetRasterBand(1).WriteArray(data)

        tif_metadata = {
            "minX": str(minx), "maxX": str(maxx),
            "minY": str(miny), "maxY": str(maxy)
        }
        dst_ds.SetMetadata(tif_metadata)

        # setting extension of output raster
        # top left x, w-e pixel resolution, rotation, top left y, rotation, n-s pixel resolution
        dst_ds.SetGeoTransform(new_transform)

        wkt = dataset.GetProjection()

        # setting spatial reference of output raster
        srs = osr.SpatialReference()
        srs.ImportFromWkt(wkt)
        dst_ds.SetProjection(srs.ExportToWkt())
        # Close output raster dataset
        dst_ds = None

        tile_num += 1

    dataset = None


if __name__ == "__main__":
    for file in os.listdir('../downloaded/tif'):
        if file.endswith(".tif"):
            split(os.path.join('../downloaded/tif', file), 10)

## Téléchargement d'images de "non terrains"

Afin de pouvoir réaliser notre classificateur, nous avions besoin de données n'étant pas des terrains. C'est ainsi que le développement d'un script de téléchargement d'images aléatoires nous est apparu nécessaire.

Notre script est relié à une API nous fournissant une image aléatoire de la même taille que nos images de la SRTM utilisées dans notre classificateur (600x600px). Il l'utilise afin de télécharger des images pour l'entraînement, la validation ainsi que pour les tests.

In [ ]:
import requests

# Training
for i in range(200):
    url = "https://picsum.photos/600/600/?random"
    response = requests.get(url)
    if response.status_code == 200:
        file_name = 'not_terrain_{}.jpg'.format(i)
        file_path = "../data/training/not_terrains/" + file_name
        with open(file_path, 'wb') as f:
            print("saving: " + file_name)
            f.write(response.content)

# Validation
for i in range(100):
    url = "https://picsum.photos/600/600/?random"
    response = requests.get(url)
    if response.status_code == 200:
        file_name = 'not_terrain_{}.jpg'.format(i)
        file_path = "../data/validation/not_terrains/" + file_name
        with open(file_path, 'wb') as f:
            print("saving: " + file_name)
            f.write(response.content)

# Test
for i in range(100):
    url = "https://picsum.photos/600/600/?random"
    response = requests.get(url)
    if response.status_code == 200:
        file_name = 'not_terrain_{}.jpg'.format(i)
        file_path = "../data/test/not_terrains/" + file_name
        with open(file_path, 'wb') as f:
            print("saving: " + file_name)
            f.write(response.content)

In [ ]:
import requests

# Training
for i in range(200):
    url = "https://picsum.photos/600/600/?random"
    response = requests.get(url)
    if response.status_code == 200:
        file_name = 'not_terrain_{}.jpg'.format(i)
        file_path = "../data/training/not_terrains/" + file_name
        with open(file_path, 'wb') as f:
            print("saving: " + file_name)
            f.write(response.content)

# Validation
for i in range(100):
    url = "https://picsum.photos/600/600/?random"
    response = requests.get(url)
    if response.status_code == 200:
        file_name = 'not_terrain_{}.jpg'.format(i)
        file_path = "../data/validation/not_terrains/" + file_name
        with open(file_path, 'wb') as f:
            print("saving: " + file_name)
            f.write(response.content)

# Test
for i in range(100):
    url = "https://picsum.photos/600/600/?random"
    response = requests.get(url)
    if response.status_code == 200:
        file_name = 'not_terrain_{}.jpg'.format(i)
        file_path = "../data/test/not_terrains/" + file_name
        with open(file_path, 'wb') as f:
            print("saving: " + file_name)
            f.write(response.content)

Enfin, après avoir extrait nos images, il convenait de les réduire de manière à pouvoir les utiliser dans notre classificateur, nous avons donc subdivisé nos images d'une taille de 6000x6000px en 100 images plus petites d'une taille de 600x600px à l'aide de la librairie GDAL permettant de faire des opérations sur des GeoTIFF.

In [ ]:
import os
from osgeo import gdal, osr


def get_extent(dataset):
    cols = dataset.RasterXSize
    rows = dataset.RasterYSize
    transform = dataset.GetGeoTransform()
    minx = transform[0]
    maxx = transform[0] + cols * transform[1] + rows * transform[2]

    miny = transform[3] + cols * transform[4] + rows * transform[5]
    maxy = transform[3]

    return {
        "minX": str(minx), "maxX": str(maxx),
        "minY": str(miny), "maxY": str(maxy),
        "cols": str(cols), "rows": str(rows)
    }


def create_tiles(minx, miny, maxx, maxy, n):
    width = maxx - minx
    height = maxy - miny

    matrix = []

    for j in range(n, 0, -1):
        for i in range(0, n):
            ulx = minx + (width / n) * i  # 10/5 * 1
            uly = miny + (height / n) * j  # 10/5 * 1

            lrx = minx + (width / n) * (i + 1)
            lry = miny + (height / n) * (j - 1)
            matrix.append([[ulx, uly], [lrx, lry]])

    return matrix


def split(file_name, n):
    raw_file_name = os.path.splitext(os.path.basename(file_name))[0].replace("_downsample", "")
    driver = gdal.GetDriverByName('GTiff')
    dataset = gdal.Open(file_name)
    band = dataset.GetRasterBand(1)
    transform = dataset.GetGeoTransform()

    extent = get_extent(dataset)

    cols = int(extent["cols"])
    rows = int(extent["rows"])

    minx = float(extent["minX"])
    maxx = float(extent["maxX"])
    miny = float(extent["minY"])
    maxy = float(extent["maxY"])

    width = maxx - minx
    height = maxy - miny

    output_path = os.path.join("../downloaded/subdivided", raw_file_name)
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    tiles = create_tiles(minx, miny, maxx, maxy, n)
    transform = dataset.GetGeoTransform()
    xOrigin = transform[0]
    yOrigin = transform[3]
    pixelWidth = transform[1]
    pixelHeight = -transform[5]

    tile_num = 1
    for tile in tiles:
        minx = tile[0][0]
        maxx = tile[1][0]
        miny = tile[1][1]
        maxy = tile[0][1]

        p1 = (minx, maxy)
        p2 = (maxx, miny)

        i1 = int((p1[0] - xOrigin) / pixelWidth)
        j1 = int((yOrigin - p1[1]) / pixelHeight)
        i2 = int((p2[0] - xOrigin) / pixelWidth)
        j2 = int((yOrigin - p2[1]) / pixelHeight)

        new_cols = i2 - i1
        new_rows = j2 - j1

        data = band.ReadAsArray(i1, j1, new_cols, new_rows)

        # print data

        new_x = xOrigin + i1 * pixelWidth
        new_y = yOrigin - j1 * pixelHeight

        new_transform = (new_x, transform[1], transform[2], new_y, transform[4], transform[5])

        output_file_base = raw_file_name + "_" + str(tile_num) + ".tif"
        output_file = os.path.join("../downloaded/subdivided", raw_file_name, output_file_base)

        dst_ds = driver.Create(output_file,
                               new_cols,
                               new_rows,
                               1,
                               gdal.GDT_Float32)

        # writting output raster
        dst_ds.GetRasterBand(1).WriteArray(data)

        tif_metadata = {
            "minX": str(minx), "maxX": str(maxx),
            "minY": str(miny), "maxY": str(maxy)
        }
        dst_ds.SetMetadata(tif_metadata)

        # setting extension of output raster
        # top left x, w-e pixel resolution, rotation, top left y, rotation, n-s pixel resolution
        dst_ds.SetGeoTransform(new_transform)

        wkt = dataset.GetProjection()

        # setting spatial reference of output raster
        srs = osr.SpatialReference()
        srs.ImportFromWkt(wkt)
        dst_ds.SetProjection(srs.ExportToWkt())
        # Close output raster dataset
        dst_ds = None

        tile_num += 1

    dataset = None


if __name__ == "__main__":
    for file in os.listdir('../downloaded/tif'):
        if file.endswith(".tif"):
            split(os.path.join('../downloaded/tif', file), 10)

Nous avons aussi développé une version permettant de les convertir directement en JPEG, les données GeoTiff n'étant pas nécessaire pour notre utilisation.

In [ ]:
from PIL import Image
import os


def split(file, chopsize, basename):
    img = Image.open(file)
    width, height = img.size
    for x0 in range(0, width, chopsize):
        for y0 in range(0, height, chopsize):
            box = (x0, y0,
                   x0 + chopsize if x0 + chopsize < width else width - 1,
                   y0 + chopsize if y0 + chopsize < height else height - 1)
            print('%s %s' % (file, box))
            img.crop(box).convert('RGB').save('../downloaded/subdivided/%s_x%03d_y%03d.jpg' % (basename, x0, y0))


if __name__ == "__main__":
    for file in os.listdir('../downloaded/tif'):
        basename = os.path.basename(file).replace('.tif', '')
        if file.endswith(".tif"):
            split(os.path.join('../downloaded/tif/', file), 600, basename)


## Téléchargement d'images de "non terrains"

Afin de pouvoir réaliser notre classificateur, nous avions besoin de données n'étant pas des terrains. C'est ainsi que le développement d'un script de téléchargement d'images aléatoires nous est apparu nécessaire.

Notre script est relié à une API nous fournissant une image aléatoire de la même taille que nos images de la SRTM utilisées dans notre classificateur (600x600px). Il l'utilise afin de télécharger des images pour l'entraînement, la validation ainsi que pour les tests.

In [ ]:
import requests

# Training
for i in range(200):
    url = "https://picsum.photos/600/600/?random"
    response = requests.get(url)
    if response.status_code == 200:
        file_name = 'not_terrain_{}.jpg'.format(i)
        file_path = "../data/training/not_terrains/" + file_name
        with open(file_path, 'wb') as f:
            print("saving: " + file_name)
            f.write(response.content)

# Validation
for i in range(100):
    url = "https://picsum.photos/600/600/?random"
    response = requests.get(url)
    if response.status_code == 200:
        file_name = 'not_terrain_{}.jpg'.format(i)
        file_path = "../data/validation/not_terrains/" + file_name
        with open(file_path, 'wb') as f:
            print("saving: " + file_name)
            f.write(response.content)

# Test
for i in range(100):
    url = "https://picsum.photos/600/600/?random"
    response = requests.get(url)
    if response.status_code == 200:
        file_name = 'not_terrain_{}.jpg'.format(i)
        file_path = "../data/test/not_terrains/" + file_name
        with open(file_path, 'wb') as f:
            print("saving: " + file_name)
            f.write(response.content)